In [23]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report,f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
from scipy.spatial.distance import cdist
import heapq
from collections import Counter
import os
import ast
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import random
import lime
import textspan
import tokenizations
random.seed(1993)
random.seed(1993)
%matplotlib inline

In [24]:
df_train = pd.read_csv("../data/span/train/train_full.tsv",sep="\t")
df_train


,spans,text
0,Correct. Enga apa military da oodi vilaiyada s...,"[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4..."
1,Dei Rajini pavam da ne varaven poraven ellam k...,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3..."
2,"Dey dey deyyy,, loosu pasangala,, munna pinna ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,Intha maari comments ku like kekuravangala ind...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
4,250 k likes ineram sila arivuketta pundaika it...,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3..."
...,...,...
4811,Ivarukkku eppodhum thalaivar kalaigner lightaa...,"[55, 56, 57, 58, 59, 60, 61, 62]"
4812,Trailer Nala irukanu oru than comment pandranu...,"[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 9..."
4813,Wigpathy Visay na Padam Flop than ithula Kabal...,"[24, 25, 26, 27, 28, 29, 102, 103, 104, 105, 1..."
4814,Vikram ella styleum set aaguthu.. Namba moonji...,"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]"


In [25]:
df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
df_test

,text,spans
0,Evanukachum pondati kolandhainu sentiment irun...,"[71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 8..."
1,Adei ennada short film la irunthu suturukinga,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]"
2,Super dialogue Oruthar mela visvasam kattrath...,"[68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 7..."
3,Epadiya jathi padam. Ponnu padama edungada. In...,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7..."
4,Ponnu mella kaivaikuravan Kai mattum illa uyir...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
...,...,...
871,காலா.காபலி.அசுரன்.பாரியேரும்.பெருமள் வந்த அப்ப...,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
872,Kekka bekka short film mathri irukey,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
873,Bayangaram... Trailerey ippadina appa Padam en...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
874,komali rasini vesam pottu tamil ilichavayangal...,"[0, 1, 2, 3, 4, 5, 6, 7, 31, 32, 33, 34, 35, 3..."


In [26]:
#BIO Data Creator

In [72]:
connll=[]
for spans,txt in zip(df_train["text"],df_train["spans"]):
    spans = ast.literal_eval(spans)
    tokens = word_tokenize(txt)
    bflag=0
    iflag=0
    txtconll=[]
    count=0
    for tk,sp in zip(tokens,textspan.get_original_spans(tokens, txt)):
#         print(sp)
        if len(sp)==0:
            continue
        exp_sp = [i for i in range(sp[0][0],sp[0][1])]
        
        if set(spans).intersection(exp_sp) and bflag==0 and iflag==0:
            bflag=1
            txtconll.append((tk,"B-CAUSE"))
#             print(tk,"B-TOKEN")
        elif set(spans).intersection(exp_sp) and bflag==1:
            iflag=1
            txtconll.append((tk,"I-CAUSE"))
#             print(tk,"I-TOKEN")
        else:                
            txtconll.append((tk,"O"))

            bflag=0
            iflag=0
        count+=1
    print(txt) 
    print(txtconll) 
    connll.append(txtconll)
    print("\n")


Correct. Enga apa military da oodi vilaiyada solli tharuvaaaayaa
[('Correct', 'O'), ('.', 'O'), ('Enga', 'O'), ('apa', 'O'), ('military', 'O'), ('da', 'O'), ('oodi', 'B-CAUSE'), ('vilaiyada', 'I-CAUSE'), ('solli', 'I-CAUSE'), ('tharuvaaaayaa', 'I-CAUSE')]


Dei Rajini pavam da ne varaven poraven ellam karithuppatha koraiya pesittu erukkan 1:58
[('Dei', 'O'), ('Rajini', 'O'), ('pavam', 'O'), ('da', 'O'), ('ne', 'O'), ('varaven', 'B-CAUSE'), ('poraven', 'I-CAUSE'), ('ellam', 'I-CAUSE'), ('karithuppatha', 'I-CAUSE'), ('koraiya', 'I-CAUSE'), ('pesittu', 'I-CAUSE'), ('erukkan', 'I-CAUSE'), ('1:58', 'O')]


Dey dey deyyy,, loosu pasangala,, munna pinna jayalalitha amma va pathrukingalada gambeeramna ennanu avanga kannula tha paakanum,, amma oda history ah ithu,, soniya gandhi mari iruku chaiiii,,, avanga bio edukanumna a-z pathutu vanthu edungadaa,, dummy akkaathinga
[('Dey', 'B-CAUSE'), ('dey', 'I-CAUSE'), ('deyyy', 'I-CAUSE'), (',', 'I-CAUSE'), (',', 'I-CAUSE'), ('loosu', 'I-CAUSE'), ('pas


Trailer paathutu padam flop nu kandu pidichavanga mattum like podunga..
[('Trailer', 'O'), ('paathutu', 'O'), ('padam', 'O'), ('flop', 'B-CAUSE'), ('nu', 'I-CAUSE'), ('kandu', 'O'), ('pidichavanga', 'O'), ('mattum', 'O'), ('like', 'O'), ('podunga', 'O'), ('..', 'O')]


Politicians ah intha padathlayum vachi senjirykanuva pola.
[('Politicians', 'O'), ('ah', 'O'), ('intha', 'O'), ('padathlayum', 'O'), ('vachi', 'B-CAUSE'), ('senjirykanuva', 'I-CAUSE'), ('pola', 'O'), ('.', 'O')]


Waste movie  waste songs  anirudh waste
[('Waste', 'B-CAUSE'), ('movie', 'I-CAUSE'), ('waste', 'I-CAUSE'), ('songs', 'I-CAUSE'), ('anirudh', 'O'), ('waste', 'B-CAUSE')]


6 th standard payan edutha Maari irukku Da Padam.
[('6', 'B-CAUSE'), ('th', 'I-CAUSE'), ('standard', 'I-CAUSE'), ('payan', 'I-CAUSE'), ('edutha', 'I-CAUSE'), ('Maari', 'I-CAUSE'), ('irukku', 'O'), ('Da', 'O'), ('Padam', 'O'), ('.', 'O')]


Pudichavan mattum like-ah podu..... pudikadhavan apdiye poidu!!!  tha...aamai badungala!!
[('Pudichavan'


RAJINI LOOKS GOOD BETTER THAN VISWASAM AJITH MAMA
[('RAJINI', 'O'), ('LOOKS', 'O'), ('GOOD', 'O'), ('BETTER', 'O'), ('THAN', 'O'), ('VISWASAM', 'O'), ('AJITH', 'O'), ('MAMA', 'B-CAUSE')]


Dai entha trailer pudekama polambura naigala...   ungala Enna solli thitturathu nu Theryala   Goatha baadu....
[('Dai', 'O'), ('entha', 'O'), ('trailer', 'O'), ('pudekama', 'O'), ('polambura', 'B-CAUSE'), ('naigala', 'I-CAUSE'), ('...', 'I-CAUSE'), ('ungala', 'O'), ('Enna', 'O'), ('solli', 'B-CAUSE'), ('thitturathu', 'I-CAUSE'), ('nu', 'I-CAUSE'), ('Theryala', 'I-CAUSE'), ('Goatha', 'O'), ('baadu', 'O'), ('....', 'O')]


Ennama arasial panringada....unga  arasialukku jaathiya use pannikiringa...
[('Ennama', 'O'), ('arasial', 'O'), ('panringada', 'O'), ('....', 'O'), ('unga', 'O'), ('arasialukku', 'B-CAUSE'), ('jaathiya', 'I-CAUSE'), ('use', 'I-CAUSE'), ('pannikiringa', 'I-CAUSE'), ('...', 'I-CAUSE')]


11m cross panniruchu Sahoo, NKP ku o .. oooo
[('11m', 'O'), ('cross', 'O'), ('panniruchu', 'O'), (



Rajani killatu Naai Arivu  ketta  jenmankal  TN Tamils🤣🤣🤣
[('Rajani', 'O'), ('killatu', 'B-CAUSE'), ('Naai', 'I-CAUSE'), ('Arivu', 'I-CAUSE'), ('ketta', 'I-CAUSE'), ('jenmankal', 'I-CAUSE'), ('TN', 'O'), ('Tamils🤣🤣🤣', 'O')]


ippo naame enna seivom hatters itku seruppaale adippom
[('ippo', 'O'), ('naame', 'O'), ('enna', 'O'), ('seivom', 'O'), ('hatters', 'O'), ('itku', 'O'), ('seruppaale', 'B-CAUSE'), ('adippom', 'I-CAUSE')]


Padam fulla night la eduthurkanga....🤔🤔🤔 oru  vela night movie ya
[('Padam', 'O'), ('fulla', 'O'), ('night', 'O'), ('la', 'O'), ('eduthurkanga', 'O'), ('....', 'O'), ('🤔🤔🤔', 'O'), ('oru', 'B-CAUSE'), ('vela', 'I-CAUSE'), ('night', 'I-CAUSE'), ('movie', 'I-CAUSE'), ('ya', 'I-CAUSE')]


Personality pathu varathu da men8tal pasangala
[('Personality', 'O'), ('pathu', 'O'), ('varathu', 'O'), ('da', 'O'), ('men8tal', 'B-CAUSE'), ('pasangala', 'I-CAUSE')]


Ena etho paithiyakara padam pol irukku ..ithula naraiya peru siriciten vera solraanuga...ithula sirikkirakku ena

Semma kaalai Da Rajini kku Periya aappu
[('Semma', 'O'), ('kaalai', 'O'), ('Da', 'O'), ('Rajini', 'O'), ('kku', 'O'), ('Periya', 'B-CAUSE'), ('aappu', 'I-CAUSE')]


Dai mokka vetri Mara unaku yethrathamanna kathaiyee eduka theriyatha!?
[('Dai', 'O'), ('mokka', 'B-CAUSE'), ('vetri', 'O'), ('Mara', 'O'), ('unaku', 'O'), ('yethrathamanna', 'B-CAUSE'), ('kathaiyee', 'I-CAUSE'), ('eduka', 'I-CAUSE'), ('theriyatha', 'I-CAUSE'), ('!', 'I-CAUSE'), ('?', 'O')]


Sunni like tha potrundha...but eppo viduvana
[('Sunni', 'O'), ('like', 'O'), ('tha', 'O'), ('potrundha', 'O'), ('...', 'O'), ('but', 'O'), ('eppo', 'B-CAUSE'), ('viduvana', 'I-CAUSE')]


Oru nalla padoo varapodhu rombo nal kalichi!
[('Oru', 'B-CAUSE'), ('nalla', 'I-CAUSE'), ('padoo', 'I-CAUSE'), ('varapodhu', 'I-CAUSE'), ('rombo', 'O'), ('nal', 'O'), ('kalichi', 'O'), ('!', 'O')]


Upadi oru trailer ku unlike potta.. Avanuku life la taste Ella... Aprum moola... Aprum arivu adhu suthuma Ella
[('Upadi', 'O'), ('oru', 'O'), ('trailer', 'O'

...

Ivan ini jayam Ravi illai komaali ravi
[('Ivan', 'O'), ('ini', 'O'), ('jayam', 'O'), ('Ravi', 'O'), ('illai', 'O'), ('komaali', 'B-CAUSE'), ('ravi', 'I-CAUSE')]


Trailer ippadi irukanum, Bigil m iruka samiii parka mudiyala
[('Trailer', 'O'), ('ippadi', 'O'), ('irukanum', 'O'), (',', 'O'), ('Bigil', 'O'), ('m', 'O'), ('iruka', 'O'), ('samiii', 'B-CAUSE'), ('parka', 'I-CAUSE'), ('mudiyala', 'I-CAUSE')]


Sathiya kolayum , anava kolayaiyum entha padam angigarikarutha ? Apadi angi garithal censor board entha padathai angigarikalama ? entha padam veliyal sathiya kolaigal melum athigarikum..
[('Sathiya', 'B-CAUSE'), ('kolayum', 'I-CAUSE'), (',', 'O'), ('anava', 'O'), ('kolayaiyum', 'O'), ('entha', 'O'), ('padam', 'O'), ('angigarikarutha', 'O'), ('?', 'O'), ('Apadi', 'O'), ('angi', 'O'), ('garithal', 'O'), ('censor', 'O'), ('board', 'O'), ('entha', 'O'), ('padathai', 'O'), ('angigarikalama', 'B-CAUSE'), ('?', 'O'), ('entha', 'O'), ('padam', 'O'), ('veliyal', 'O'), ('sathiya', 'B-CAUSE'), ('k

you can kill a man, but you can't kill Ambedkar Periyar Marx
[('you', 'O'), ('can', 'O'), ('kill', 'B-CAUSE'), ('a', 'I-CAUSE'), ('man', 'I-CAUSE'), (',', 'I-CAUSE'), ('but', 'O'), ('you', 'O'), ('ca', 'O'), ("n't", 'O'), ('kill', 'O'), ('Ambedkar', 'O'), ('Periyar', 'O'), ('Marx', 'O')]


Enna da ithu trailer ah Serial ah
[('Enna', 'O'), ('da', 'O'), ('ithu', 'O'), ('trailer', 'B-CAUSE'), ('ah', 'I-CAUSE'), ('Serial', 'I-CAUSE'), ('ah', 'I-CAUSE')]


Samuthirakani padathula Vara Vara athulya nadichikite irukaa....something wrong
[('Samuthirakani', 'O'), ('padathula', 'O'), ('Vara', 'O'), ('Vara', 'O'), ('athulya', 'B-CAUSE'), ('nadichikite', 'I-CAUSE'), ('irukaa', 'I-CAUSE'), ('....', 'I-CAUSE'), ('something', 'O'), ('wrong', 'O')]


Kulandhaigal kappagam la kuda indha padam odaadhu
[('Kulandhaigal', 'O'), ('kappagam', 'O'), ('la', 'O'), ('kuda', 'O'), ('indha', 'B-CAUSE'), ('padam', 'I-CAUSE'), ('odaadhu', 'I-CAUSE')]


Ne nala actor than othukura ana thalapathy kuda poti podra alavu


anils oda katharal sattham athigama irukay
[('anils', 'B-CAUSE'), ('oda', 'I-CAUSE'), ('katharal', 'I-CAUSE'), ('sattham', 'I-CAUSE'), ('athigama', 'O'), ('irukay', 'O')]


Thala ~ Thalapathy rendu perium Ooititaan
[('Thala', 'O'), ('~', 'O'), ('Thalapathy', 'O'), ('rendu', 'O'), ('perium', 'O'), ('Ooititaan', 'B-CAUSE')]


Indha music yenagayo keta madhiri iruku... Anybody know this bgm
[('Indha', 'O'), ('music', 'O'), ('yenagayo', 'B-CAUSE'), ('keta', 'I-CAUSE'), ('madhiri', 'I-CAUSE'), ('iruku', 'I-CAUSE'), ('...', 'I-CAUSE'), ('Anybody', 'O'), ('know', 'O'), ('this', 'O'), ('bgm', 'O')]


Rajini arasiyal vaalkaiya ore varthai la senjutanga
[('Rajini', 'O'), ('arasiyal', 'O'), ('vaalkaiya', 'O'), ('ore', 'B-CAUSE'), ('varthai', 'I-CAUSE'), ('la', 'I-CAUSE'), ('senjutanga', 'I-CAUSE')]


Dislike potavan ellam sam anderson fans.....
[('Dislike', 'O'), ('potavan', 'O'), ('ellam', 'O'), ('sam', 'B-CAUSE'), ('anderson', 'I-CAUSE'), ('fans', 'I-CAUSE'), ('.....', 'I-CAUSE')]


Mgr kalain

Indha trailer pudikalana avan tamil cinema rasigan Illa. ... wokka makka. .... therikkudhu
[('Indha', 'O'), ('trailer', 'O'), ('pudikalana', 'O'), ('avan', 'B-CAUSE'), ('tamil', 'I-CAUSE'), ('cinema', 'I-CAUSE'), ('rasigan', 'I-CAUSE'), ('Illa', 'I-CAUSE'), ('.', 'I-CAUSE'), ('...', 'O'), ('wokka', 'O'), ('makka', 'B-CAUSE'), ('.', 'I-CAUSE'), ('....', 'O'), ('therikkudhu', 'O')]


Dai  potai vetri maran unaku  night  thavira pagala  edakathriyadha
[('Dai', 'B-CAUSE'), ('potai', 'I-CAUSE'), ('vetri', 'I-CAUSE'), ('maran', 'O'), ('unaku', 'O'), ('night', 'O'), ('thavira', 'O'), ('pagala', 'B-CAUSE'), ('edakathriyadha', 'I-CAUSE')]


tun tun aunty maari iruku...       ena da pani vatchirukinga...
[('tun', 'B-CAUSE'), ('tun', 'I-CAUSE'), ('aunty', 'I-CAUSE'), ('maari', 'I-CAUSE'), ('iruku', 'I-CAUSE'), ('...', 'I-CAUSE'), ('ena', 'I-CAUSE'), ('da', 'I-CAUSE'), ('pani', 'I-CAUSE'), ('vatchirukinga', 'I-CAUSE'), ('...', 'I-CAUSE')]


vijay konjam moolaiya kozhambittu overa act pannitaaaaaaa


Viswasamnala dhan petta trending ke vandhurukku
[('Viswasamnala', 'O'), ('dhan', 'O'), ('petta', 'O'), ('trending', 'B-CAUSE'), ('ke', 'I-CAUSE'), ('vandhurukku', 'I-CAUSE')]


Velaiya poyi paarungada.....intha movie confirm flap
[('Velaiya', 'O'), ('poyi', 'O'), ('paarungada', 'O'), ('.....', 'O'), ('intha', 'O'), ('movie', 'O'), ('confirm', 'B-CAUSE'), ('flap', 'I-CAUSE')]


Just looks like cartoon so bad
[('Just', 'O'), ('looks', 'B-CAUSE'), ('like', 'I-CAUSE'), ('cartoon', 'I-CAUSE'), ('so', 'O'), ('bad', 'O')]


Kandipa padam flop reason soori 🤣🤣🤣and the great villan pakara apo ve sema bayama iruku
[('Kandipa', 'O'), ('padam', 'O'), ('flop', 'B-CAUSE'), ('reason', 'I-CAUSE'), ('soori', 'O'), ('🤣🤣🤣and', 'O'), ('the', 'O'), ('great', 'O'), ('villan', 'O'), ('pakara', 'O'), ('apo', 'O'), ('ve', 'O'), ('sema', 'O'), ('bayama', 'O'), ('iruku', 'O')]


Jaathi jaathi nu sollitu saagarappa ena kondu poga poreenga ellaarum... Ovoruthanukku pinnaadiyum ovoru jaathi kootam laam irukkum.. at

In [73]:
#Simple transformer format

train_data=[]
for index,fm in enumerate(connll):
    tmp=[]
#     print(fm)
    for pair in fm:
#         print(pair)
        train_data.append([index,pair[0],pair[1]])

In [74]:
train_data

[[0, 'Correct', 'O'],
 [0, '.', 'O'],
 [0, 'Enga', 'O'],
 [0, 'apa', 'O'],
 [0, 'military', 'O'],
 [0, 'da', 'O'],
 [0, 'oodi', 'B-CAUSE'],
 [0, 'vilaiyada', 'I-CAUSE'],
 [0, 'solli', 'I-CAUSE'],
 [0, 'tharuvaaaayaa', 'I-CAUSE'],
 [1, 'Dei', 'O'],
 [1, 'Rajini', 'O'],
 [1, 'pavam', 'O'],
 [1, 'da', 'O'],
 [1, 'ne', 'O'],
 [1, 'varaven', 'B-CAUSE'],
 [1, 'poraven', 'I-CAUSE'],
 [1, 'ellam', 'I-CAUSE'],
 [1, 'karithuppatha', 'I-CAUSE'],
 [1, 'koraiya', 'I-CAUSE'],
 [1, 'pesittu', 'I-CAUSE'],
 [1, 'erukkan', 'I-CAUSE'],
 [1, '1:58', 'O'],
 [2, 'Dey', 'B-CAUSE'],
 [2, 'dey', 'I-CAUSE'],
 [2, 'deyyy', 'I-CAUSE'],
 [2, ',', 'I-CAUSE'],
 [2, ',', 'I-CAUSE'],
 [2, 'loosu', 'I-CAUSE'],
 [2, 'pasangala', 'I-CAUSE'],
 [2, ',', 'I-CAUSE'],
 [2, ',', 'O'],
 [2, 'munna', 'O'],
 [2, 'pinna', 'O'],
 [2, 'jayalalitha', 'O'],
 [2, 'amma', 'O'],
 [2, 'va', 'O'],
 [2, 'pathrukingalada', 'B-CAUSE'],
 [2, 'gambeeramna', 'O'],
 [2, 'ennanu', 'O'],
 [2, 'avanga', 'O'],
 [2, 'kannula', 'O'],
 [2, 'tha', 'O'],


In [76]:
train_df_ner= pd.DataFrame(train_data, columns=["sentence_id", "words", "labels"])
train_df_ner.to_csv("../data/bio/train_ner.tsv",sep="\t",index=None)

In [77]:
# Test data Prep

In [78]:
connll=[]
for txt,spans in zip(df_test["text"],df_test["spans"]):
    spans = ast.literal_eval(spans)
    tokens = word_tokenize(txt)
    bflag=0
    iflag=0
    txtconll=[]
    count=0
    for tk,sp in zip(tokens,textspan.get_original_spans(tokens, txt)):
#         print(sp)
        if len(sp)==0:
            continue
        exp_sp = [i for i in range(sp[0][0],sp[0][1])]
        
        if set(spans).intersection(exp_sp) and bflag==0 and iflag==0:
            bflag=1
            txtconll.append((tk,"B-CAUSE"))
#             print(tk,"B-TOKEN")
        elif set(spans).intersection(exp_sp) and bflag==1:
            iflag=1
            txtconll.append((tk,"I-CAUSE"))
#             print(tk,"I-TOKEN")
        else:                
            txtconll.append((tk,"O"))

            bflag=0
            iflag=0
        count+=1
    print(txt) 
    print(txtconll) 
    connll.append(txtconll)
    print("\n")


Evanukachum pondati kolandhainu sentiment irundha , apdiyaeoadi poairu, kola gandula iruken mavana kollama vida mata
[('Evanukachum', 'O'), ('pondati', 'O'), ('kolandhainu', 'O'), ('sentiment', 'O'), ('irundha', 'O'), (',', 'O'), ('apdiyaeoadi', 'O'), ('poairu', 'O'), (',', 'O'), ('kola', 'B-CAUSE'), ('gandula', 'I-CAUSE'), ('iruken', 'I-CAUSE'), ('mavana', 'I-CAUSE'), ('kollama', 'I-CAUSE'), ('vida', 'I-CAUSE'), ('mata', 'I-CAUSE')]


Adei ennada short film la irunthu suturukinga
[('Adei', 'O'), ('ennada', 'O'), ('short', 'O'), ('film', 'O'), ('la', 'O'), ('irunthu', 'O'), ('suturukinga', 'B-CAUSE')]


Super dialogue  Oruthar mela visvasam kattrathukkaga innoruthavarai asinga paduthathinga
[('Super', 'O'), ('dialogue', 'O'), ('Oruthar', 'O'), ('mela', 'O'), ('visvasam', 'O'), ('kattrathukkaga', 'O'), ('innoruthavarai', 'O'), ('asinga', 'B-CAUSE'), ('paduthathinga', 'I-CAUSE')]


Epadiya jathi padam. Ponnu padama edungada. India vallarasu agidun. Pogada nigalun unga jathi padam. Anda p



Nive  kutty aa enga da ??
[('Nive', 'O'), ('kutty', 'O'), ('aa', 'O'), ('enga', 'B-CAUSE'), ('da', 'I-CAUSE'), ('?', 'I-CAUSE'), ('?', 'O')]


Oru mokka padatha eduthutu Nadodigal sequel nu sonna nambiduvoma !! Dei yaara yemaatha paakura ??
[('Oru', 'O'), ('mokka', 'B-CAUSE'), ('padatha', 'I-CAUSE'), ('eduthutu', 'O'), ('Nadodigal', 'O'), ('sequel', 'O'), ('nu', 'O'), ('sonna', 'O'), ('nambiduvoma', 'O'), ('!', 'O'), ('!', 'O'), ('Dei', 'B-CAUSE'), ('yaara', 'I-CAUSE'), ('yemaatha', 'I-CAUSE'), ('paakura', 'I-CAUSE'), ('?', 'O'), ('?', 'O')]


Vandi reverse edutha oru tone varumey andha mari itukuu ,andha mari nu ila adhey dha
[('Vandi', 'O'), ('reverse', 'O'), ('edutha', 'O'), ('oru', 'O'), ('tone', 'O'), ('varumey', 'O'), ('andha', 'B-CAUSE'), ('mari', 'I-CAUSE'), ('itukuu', 'I-CAUSE'), (',', 'O'), ('andha', 'O'), ('mari', 'O'), ('nu', 'O'), ('ila', 'O'), ('adhey', 'O'), ('dha', 'O')]


FABulous trailer yaar... semma.. Rip Van Winkle
[('FABulous', 'O'), ('trailer', 'O'), ('yaar', '

ennaiya ella gangster ayum padathula kaatreenga
[('ennaiya', 'O'), ('ella', 'B-CAUSE'), ('gangster', 'I-CAUSE'), ('ayum', 'I-CAUSE'), ('padathula', 'I-CAUSE'), ('kaatreenga', 'I-CAUSE')]


Indha padathukku  dislike poduravanga Manisa jenmamay illa
[('Indha', 'O'), ('padathukku', 'O'), ('dislike', 'O'), ('poduravanga', 'O'), ('Manisa', 'B-CAUSE'), ('jenmamay', 'I-CAUSE'), ('illa', 'I-CAUSE')]


Samoothrakani edukkura nadikkira  padathulalam caste kondu vanthu thinuchidraru.....  confirm padam flop...
[('Samoothrakani', 'O'), ('edukkura', 'O'), ('nadikkira', 'O'), ('padathulalam', 'O'), ('caste', 'O'), ('kondu', 'B-CAUSE'), ('vanthu', 'I-CAUSE'), ('thinuchidraru', 'I-CAUSE'), ('.....', 'I-CAUSE'), ('confirm', 'I-CAUSE'), ('padam', 'I-CAUSE'), ('flop', 'I-CAUSE'), ('...', 'I-CAUSE')]


Vetrimaran padatha patha saathi adakumurai innum iruka mari katuvan. Dont support him
[('Vetrimaran', 'O'), ('padatha', 'O'), ('patha', 'O'), ('saathi', 'O'), ('adakumurai', 'O'), ('innum', 'B-CAUSE'), ('ir

In [79]:
len(connll)

876

In [82]:
#Simple transformer format

test_data=[]
for index,fm in enumerate(connll):
    for pair in fm:
        test_data.append([index,pair[0],pair[1]])

test_df_ner= pd.DataFrame(test_data, columns=["sentence_id", "words", "labels"])
test_df_ner.to_csv("../data/bio/test_ner.tsv",sep="\t",index=None)

In [83]:
test_df_ner

,sentence_id,words,labels
0,0,Evanukachum,O
1,0,pondati,O
2,0,kolandhainu,O
3,0,sentiment,O
4,0,irundha,O
...,...,...,...
11198,875,ungala,O
11199,875,thanda,O
11200,875,soldraru,O
11201,875,amai,B-CAUSE
